In [2]:
import datetime
import random
import logging

import requests
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, Date, text
from sqlalchemy.exc import SQLAlchemyError
from taipy.gui import Gui, notify

import json
import os


In [3]:
class DatabaseManager:
    def __init__(self, db_url):
        self.db_url = db_url
        self.engine = create_engine(db_url)
        self.metadata = MetaData()
        self._initialize_tables()

    def _initialize_tables(self):
        # Define and create health_metrics table if not exists
        health_metrics_table = Table(
            'health_metrics', self.metadata,
            Column('date', Date, primary_key=True),
            Column('sleep_hours', Float),
            Column('calorie_expenditure', Integer),
            Column('weight', Float),
            Column('calorie_intake', Integer),
            Column('fasting_hours', Float),
            Column('daily_lifescore', Float)
        )
        self.metadata.create_all(self.engine)

    def insert_data(self, df, table_name='health_metrics'):
        try:
            df.to_sql(table_name, con=self.engine, if_exists='append', index=False)
            print(f"Data inserted successfully into the '{table_name}' table.")
        except SQLAlchemyError as e:
            print(f"An error occurred while inserting data: {e}")

    def fetch_data(self, query):
        try:
            with self.engine.connect() as connection:
                return pd.read_sql(query, con=connection)
        except SQLAlchemyError as e:
            print(f"An error occurred while fetching data: {e}")
            return pd.DataFrame()

    def upsert_data(self, df, table_name='health_metrics'):
        """
        Upserts a given DataFrame into the health_metrics table in the database.
        This function keeps 'None' values as NULL in the database.
        
        Args:
            df (pd.DataFrame): DataFrame with columns: 'date', 'calorie_expenditure', 'sleep_hours', 'weight', 
                            'calorie_intake', 'fasting_hours', 'daily_lifescore'.
        """
        # Convert columns to appropriate types, ensuring they are converted to Python native types
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['calorie_expenditure'] = df['calorie_expenditure'].replace({np.nan: None}).astype('Int64')
        df['calorie_intake'] = df['calorie_intake'].replace({np.nan: None}).astype('Int64')
        df['weight'] = df['weight'].replace({np.nan: None}).astype(float)
        df['sleep_hours'] = df['sleep_hours'].replace({np.nan: None}).astype(float)
        df['fasting_hours'] = df['fasting_hours'].replace({np.nan: None}).astype(float)
        df['daily_lifescore'] = df['daily_lifescore'].replace({np.nan: None}).astype(float)

        # Using SQLAlchemy engine to establish connection and perform upsert
        try:
            with self.engine.connect() as connection:
                # Prepare the SQL upsert statement
                sql = """
                INSERT INTO health_metrics (date, sleep_hours, calorie_expenditure, weight, calorie_intake, fasting_hours, daily_lifescore)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (date) DO UPDATE SET
                    weight = EXCLUDED.weight,
                    calorie_intake = EXCLUDED.calorie_intake,
                    fasting_hours = EXCLUDED.fasting_hours,
                    daily_lifescore = EXCLUDED.daily_lifescore;
                """
                
                # Iterate over each row in the DataFrame and upsert each row
                for _, row in df.iterrows():
                    row_to_insert = tuple(row)
                    # Execute the SQL command
                    connection.execute(sql, row_to_insert)

                print("Data upserted successfully into 'health_metrics'.")

        except SQLAlchemyError as e:
            print(f"An error occurred while upserting data: {e}")


In [4]:


class FitbitAPIClient:
    def __init__(self, access_token, user_id='5FR3J5'):
        self.base_url = 'https://api.fitbit.com'
        self.user_id = user_id
        self.access_token = access_token
        self.headers = {"Authorization": f"Bearer {self.access_token}"}

    def fetch_sleep_data(self, start_date, end_date):
        url = f"{self.base_url}/1.2/user/{self.user_id}/sleep/date/{start_date}/{end_date}.json"
        return self._fetch_data(url, "sleep")

    def fetch_calorie_expenditure_data(self, start_date, end_date):
        url = f"{self.base_url}/1.2/user/{self.user_id}/activities/calories/date/{start_date}/{end_date}.json"
        return self._fetch_data(url, "activities-calories")

    def _fetch_data(self, url, key):
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json().get(key, None)
        else:
            logging.error(f"Failed to fetch data: {response.text}")
            return None


In [5]:

class DataManager:
    def __init__(self):
        pass

    def process_fetched_data(self, calorie_data, sleep_data):
        # Process fetched data into a DataFrame
        df_manualextraction = self._create_calorie_dataframe(calorie_data)
        df_sleep = self._create_sleep_dataframe(sleep_data)

        # Merge and further process
        if not df_sleep.empty:
            df_manualextraction = pd.merge(df_manualextraction, df_sleep, on='date', how='left')
            df_manualextraction = df_manualextraction.sort_values(by='date').reset_index(drop=True)

        # Add additional columns for analysis
        df_manualextraction['weight'] = [round(random.uniform(50, 55), 1) for _ in range(len(df_manualextraction))]
        df_manualextraction['calorie_intake'] = [random.randint(1600, 2000) for _ in range(len(df_manualextraction))]
        df_manualextraction['fasting_hours'] = [round(random.uniform(0, 16), 2) for _ in range(len(df_manualextraction))]
        df_manualextraction['daily_lifescore'] = None
        return df_manualextraction

    def _create_calorie_dataframe(self, calorie_data):
        calorie_dates = [entry['dateTime'] for entry in calorie_data]
        calorie_expenditure_list = [int(entry['value']) for entry in calorie_data]

        df_manualextraction = pd.DataFrame({
            'date': calorie_dates,
            'calorie_expenditure': calorie_expenditure_list
        })
        df_manualextraction['date'] = pd.to_datetime(df_manualextraction['date'])
        return df_manualextraction

    def _create_sleep_dataframe(self, sleep_data):
        sleep_records = [{'date': record['dateOfSleep'], 'sleep_minutes': record['minutesAsleep']} for record in sleep_data]
        df_sleep = pd.DataFrame(sleep_records)

        if not df_sleep.empty:
            df_sleep['date'] = pd.to_datetime(df_sleep['date'])
            df_sleep = df_sleep.groupby('date', as_index=False).sum()
            df_sleep['sleep_hours'] = round(df_sleep['sleep_minutes'] / 60, 2)
            return df_sleep[['date', 'sleep_hours']]
        return pd.DataFrame()



In [6]:
def submit_data(state):
    try:
        with engine.connect() as connection:
            entry_date_str = state.entry_date.strftime("%Y-%m-%d")
            query = text("SELECT calorie_expenditure, sleep_hours FROM health_metrics WHERE date = :date_param")
            result = connection.execute(query, {"date_param": entry_date_str}).fetchone()

            # Extract results or set to None if no data found
            calorie_expenditure, sleep_hours = (result if result else (None, None))

            # Create a new row and overwrite the DataFrame
            new_entry = pd.DataFrame([{
                'date': state.entry_date,
                'calorie_expenditure': calorie_expenditure,
                'sleep_hours': sleep_hours,
                'weight': state.weight,
                'calorie_intake': state.calorie_intake,
                'fasting_hours': state.fasting_hours,
                'daily_lifescore': float('nan')
            }])
            state.df_entry = new_entry

            # Update the GUI to reflect changes in df_entry
            state.df_entry = state.df_entry.copy()

            # Print updated DataFrame
            print(state.df_entry)

            # Call the upsert function with the new DataFrame
            upsert_health_metrics(state.df_entry)

    except Exception as e:
        print(f"An error occurred: {e}")

# Fetch existing data for display in charts
def fetch_health_data():
    try:
        with engine.connect() as connection:
            health_data = pd.read_sql('SELECT * FROM health_metrics ORDER BY date ASC', con=connection)
            health_data['date'] = health_data['date'].apply(lambda x: x.strftime('%Y-%m-%d'))  # Convert date to string
            health_data["calorie_deficit"] = health_data["calorie_expenditure"] - health_data["calorie_intake"]
            return health_data
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return pd.DataFrame()

# Initial state variables
weight = 50.0
entry_date = datetime.date.today()
calorie_intake = 2000
fasting_hours = 16
df_entry = pd.DataFrame(columns=["date", "calorie_expenditure", "sleep_hours", "weight", "calorie_intake", "fasting_hours", "daily_lifescore"])
df_fetcheddata = fetch_health_data()

# Define the page layout with inputs and a submit button
page = """
<center><h1 style="color:#ADD8E6;">Health Tracking Dashboard</h1></center>
<|layout|columns=2|gap=20px|>
<|
<h3>Enter Entry Date</h3>
<|{entry_date}|date|label=Select a Date|>

<h3>Enter Weight (kg)</h3>
<|{weight}|input|type=number|label=Enter Weight (kg)|>

<h3>Enter Calorie Intake</h3>
<|{calorie_intake}|input|type=number|label=Enter Calorie Intake (kcal)|>

<h3>Enter Fasting Hours</h3>
<|{fasting_hours}|input|type=number|label=Enter Fasting Hours|>

<|Submit|button|on_action=submit_data|class_name=button|>
|>

<|layout|columns=1|gap=10px|>
<|>
<center><h2 style="color:#87CEEB;">Weight Over Time</h2></center>
<|{df_fetcheddata}|chart|type=line|x=date|y=weight|color=#4682B4|title="Weight Tracking"|>
|>

<|>
<center><h2 style="color:#87CEEB;">Calorie Intake vs. Expenditure</h2></center>
<|{df_fetcheddata}|chart|type=bar|x=date|y[1]=calorie_intake|y[2]=calorie_expenditure|color[1]=#6495ED|color[2]=#4169E1|title="Calorie Metrics"|>
|>

<|>
<center><h2 style="color:#87CEEB;">Calorie Deficit Over Time</h2></center>
<|{df_fetcheddata}|chart|type=line|x=date|y=calorie_deficit|color=#1E90FF|title="Calorie Deficit Tracking"|>
|>

<|>
<center><h2 style="color:#87CEEB;">Sleep Hours Over Time</h2></center>
<|{df_fetcheddata}|chart|type=line|x=date|y=sleep_hours|color=#00BFFF|title="Sleep Hours"|>
|>
"""

# Create the GUI instance with initial variables in the state
gui = Gui(page=page)

# Run the GUI only if the script is executed directly
if __name__ == "__main__":
    gui.run(
        title="Health Tracking Input Example",
        dark_mode=True,
        port=5001,
        state={
            "weight": weight,
            "entry_date": entry_date,
            "calorie_intake": calorie_intake,
            "fasting_hours": fasting_hours,
            "df_entry": df_entry,
            "df_fetcheddata": df_fetcheddata
        }
    )

An error occurred while fetching data: name 'engine' is not defined
[2024-11-04 19:42:31][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2024-11-04 19:42:36][Taipy][INFO]  * Server starting on http://127.0.0.1:5001


c:\Users\aksha\AppData\Local\Programs\Python\Python312\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


--- 9 warning(s) were found for page '/' in variable 'page' ---
 - Warning 1: Error column "date" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 2: Error column "sleep_hours" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 3: Error column "date" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 4: Error column "calorie_deficit" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 5: Error column "calorie_intake" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 6: Error column "date" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 7: Error column "calorie_expenditure" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 8: Error column "weight" is not present in the Dataframe "Empty DataFrame
Columns: []
Index: []".
 - Warning 9: Error column "date" i